In [1]:
import yfinance as yf
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import snapshot_download
import datetime

# === CONFIG ===
SYMBOL      = "NVDA"
INTERVAL    = "5m"
PERIOD      = "1d"
CONF_THRESH = 0.75
REPO_ID     = "TheBloke/Llama-7B-4bit-GGUF"  # quantized Llama-7B example

# === 1. Dynamically download & load the model ===
# This will cache the repo to ~/.cache/huggingface/hub by default
cache_dir = snapshot_download(repo_id=REPO_ID)

tokenizer = AutoTokenizer.from_pretrained(cache_dir, use_fast=False)
model     = AutoModelForCausalLM.from_pretrained(
    cache_dir,
    torch_dtype=torch.float16,
    device_map="auto"
)
model.eval()

def query_local_llm(prompt: str, max_tokens: int = 128) -> str:
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        temperature=0.0,
        top_p=0.9,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )
    # Decode only the newly generated tokens
    return tokenizer.decode(
        output[0][inputs.input_ids.shape[-1]:],
        skip_special_tokens=True
    ).strip()

# === 2. Fetch headline sentiment ===
def fetch_sentiment(symbol: str, n_headlines: int = 5):
    url  = f"https://finance.yahoo.com/quote/{symbol}"
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, "html.parser")
    # Selector may change; this grabs the first few headline links
    head_elems = soup.select(".Mb\\(5px\\) .Fw\\(600\\) a")[:n_headlines]
    headlines  = [h.get_text() for h in head_elems]
    polarities = [TextBlob(h).sentiment.polarity for h in headlines] or [0]
    return sum(polarities) / len(polarities), headlines

# === 3. Main trading logic ===
def main():
    # --- Market data & EMAs ---
    data   = yf.Ticker(SYMBOL).history(period=PERIOD, interval=INTERVAL)
    latest = data.iloc[-1]
    ema20  = data["Close"].ewm(span=20).mean().iloc[-1]
    ema50  = data["Close"].ewm(span=50).mean().iloc[-1]

    # --- Sentiment ---
    avg_sent, headlines = fetch_sentiment(SYMBOL)

    # --- Build prompt ---
    snapshot = {
        "time":      str(datetime.datetime.now()),
        "price":     {
            "open":  round(latest.Open,  2),
            "high":  round(latest.High,  2),
            "low":   round(latest.Low,   2),
            "close": round(latest.Close, 2)
        },
        "EMA20":     round(ema20, 2),
        "EMA50":     round(ema50, 2),
        "sentiment": round(avg_sent, 2),
        "headlines": headlines
    }

    prompt = f"""
You are a disciplined intraday trading assistant. Here’s the current snapshot for {SYMBOL}:

{snapshot}

Based on EMA20 vs EMA50 and the average sentiment of recent headlines:
— Should we ENTER a trade now? (Answer LONG, SHORT, or NO ENTRY)
— Provide a confidence score between 0.0 and 1.0.

Respond in strict JSON: {{ "signal":"", "confidence":0.00 }}
"""

    # --- Query the local LLM ---
    llm_output = query_local_llm(prompt)
    # Expect something like: {"signal":"LONG","confidence":0.82}
    result = eval(llm_output)

    signal, conf = result["signal"], result["confidence"]
    price = latest.Close

    # --- Execution rule ---
    if signal == "LONG" and conf > CONF_THRESH and ema20 > ema50:
        print(f"➡️ ENTER LONG {SYMBOL} @ {price} (conf={conf:.2f})")
    elif signal == "SHORT" and conf > CONF_THRESH and ema20 < ema50:
        print(f"➡️ ENTER SHORT {SYMBOL} @ {price} (conf={conf:.2f})")
    else:
        print("➡️ NO ENTRY")

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'textblob'